# Child Welfare Episodes - Exploratory Data Analysis
## Chapin Hall Capstone Project

This notebook explores patterns in child welfare episodes, focusing on:
- Length of stay distributions
- Demographic disparities
- Placement stability
- Long-stay risk factors

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Libraries imported successfully!')

In [ ]:
# Connect to PostgreSQL
engine = create_engine('postgresql://postgres@localhost:5432/chapinhall_capstone')

# Test connection
with engine.connect() as conn:
    result = conn.execute('SELECT COUNT(*) FROM children')
    count = result.scalar()
    print(f'✓ Connected to database')
    print(f'✓ Found {count} children in database')

## 1. Load Data from Analysis Views

In [ ]:
# Load the main analysis view
query = "SELECT * FROM child_episode"
df = pd.read_sql(query, engine)

print(f"Loaded {len(df)} episodes")
print(f"Columns: {df.shape[1]}")
df.head()

In [ ]:
# Load summary metrics
metrics = pd.read_sql("SELECT * FROM metrics_summary", engine)
metrics.T

## 2. Basic Statistics

In [ ]:
# Episode length statistics
print("Length of Stay Statistics (days)")
print("="*50)
print(df['episode_length_days'].describe())
print(f"\nLong stay (3+ years): {df['long_stay_3yr'].sum()} ({df['long_stay_3yr'].mean()*100:.1f}%)")
print(f"Long stay (2+ years): {df['long_stay_2yr'].sum()} ({df['long_stay_2yr'].mean()*100:.1f}%)")

In [ ]:
# Age at entry statistics
print("Age at Entry Statistics (years)")
print("="*50)
print(df['age_at_entry_years'].describe())

## 3. Visualizations

In [ ]:
# Distribution of episode length
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['episode_length_days'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['episode_length_days'].median(), color='red', 
                linestyle='--', label=f'Median: {df["episode_length_days"].median():.0f} days')
axes[0].set_xlabel('Episode Length (days)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Episode Length')
axes[0].legend()

# Box plot by long stay status
df.boxplot(column='episode_length_days', by='long_stay_3yr', ax=axes[1])
axes[1].set_xlabel('Long Stay (3+ years)')
axes[1].set_ylabel('Episode Length (days)')
axes[1].set_title('Episode Length by Long Stay Status')

plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
# Age distribution at entry
plt.figure(figsize=(12, 5))
plt.hist(df['age_at_entry_years'], bins=18, edgecolor='black', alpha=0.7)
plt.xlabel('Age at Entry (years)')
plt.ylabel('Number of Episodes')
plt.title('Age Distribution at Entry into Care')
plt.axvline(df['age_at_entry_years'].median(), color='red', 
            linestyle='--', label=f'Median: {df["age_at_entry_years"].median():.1f} years')
plt.legend()
plt.show()

## 4. Disparities Analysis

In [ ]:
# Long stay rates by race
race_analysis = df.groupby('race').agg({
    'episode_id': 'count',
    'long_stay_3yr': 'mean',
    'episode_length_days': 'median'
}).round(3)

race_analysis.columns = ['Total Episodes', 'Long Stay Rate', 'Median Length (days)']
race_analysis['Long Stay Rate'] = (race_analysis['Long Stay Rate'] * 100).round(1)
race_analysis = race_analysis.sort_values('Long Stay Rate', ascending=False)

print("Long Stay Rates by Race")
print("="*60)
print(race_analysis)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
race_analysis['Long Stay Rate'].plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_ylabel('Long Stay Rate (%)')
ax.set_title('Long Stay Rates (3+ years) by Race')
ax.set_xlabel('Race')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# County comparison
county_analysis = df.groupby('initial_county').agg({
    'episode_id': 'count',
    'long_stay_3yr': 'mean',
    'episode_length_days': 'median',
    'age_at_entry_years': 'mean'
}).round(2)

county_analysis.columns = ['Episodes', 'Long Stay Rate', 'Median Length', 'Avg Entry Age']
county_analysis['Long Stay Rate'] = (county_analysis['Long Stay Rate'] * 100).round(1)
county_analysis = county_analysis.sort_values('Episodes', ascending=False)

print("\nCounty Comparison")
print("="*60)
print(county_analysis)

## 5. Trends Over Time

In [ ]:
# Entries by year
yearly_entries = df.groupby('entry_year').agg({
    'episode_id': 'count',
    'long_stay_3yr': 'mean'
}).round(3)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Entries over time
yearly_entries['episode_id'].plot(kind='line', marker='o', ax=axes[0])
axes[0].set_xlabel('Entry Year')
axes[0].set_ylabel('Number of Entries')
axes[0].set_title('Entries into Care by Year')
axes[0].grid(True, alpha=0.3)

# Long stay rate over time
(yearly_entries['long_stay_3yr'] * 100).plot(kind='line', marker='o', ax=axes[1], color='coral')
axes[1].set_xlabel('Entry Year')
axes[1].set_ylabel('Long Stay Rate (%)')
axes[1].set_title('Long Stay Rate by Entry Year')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Removal Reasons Analysis

In [ ]:
# Top removal reasons
removal_reasons = df['removal_reason'].value_counts()

plt.figure(figsize=(12, 6))
removal_reasons.plot(kind='barh', color='teal', edgecolor='black')
plt.xlabel('Number of Episodes')
plt.ylabel('Removal Reason')
plt.title('Most Common Removal Reasons')
plt.tight_layout()
plt.show()

# Long stay rate by removal reason
reason_longstay = df.groupby('removal_reason')['long_stay_3yr'].mean().sort_values(ascending=False) * 100
print("\nLong Stay Rate by Removal Reason")
print("="*50)
print(reason_longstay.round(1))

## 7. Goal Achievement Analysis

In [ ]:
# Distribution of goals
goal_dist = df['goal'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart of goals
goal_dist.plot(kind='pie', ax=axes[0], autopct='%1.1f%%')
axes[0].set_ylabel('')
axes[0].set_title('Distribution of Permanency Goals')

# Long stay rate by goal
goal_longstay = df.groupby('goal')['long_stay_3yr'].mean().sort_values(ascending=False) * 100
goal_longstay.plot(kind='barh', ax=axes[1], color='salmon', edgecolor='black')
axes[1].set_xlabel('Long Stay Rate (%)')
axes[1].set_ylabel('Goal')
axes[1].set_title('Long Stay Rate by Permanency Goal')

plt.tight_layout()
plt.show()

## 8. Key Findings Summary

In [ ]:
print("KEY FINDINGS")
print("="*70)
print(f"\n1. CASELOAD")
print(f"   - Total children: {df['child_id'].nunique()}")
print(f"   - Total episodes: {len(df)}")
print(f"   - Active episodes: {df['is_active'].sum()}")

print(f"\n2. LENGTH OF STAY")
print(f"   - Median: {df['episode_length_days'].median():.0f} days ({df['episode_length_years'].median():.1f} years)")
print(f"   - Mean: {df['episode_length_days'].mean():.0f} days ({df['episode_length_years'].mean():.1f} years)")
print(f"   - Long stay rate (3+ yrs): {df['long_stay_3yr'].mean()*100:.1f}%")

print(f"\n3. DEMOGRAPHICS")
print(f"   - Most common race: {df['race'].mode()[0]}")
print(f"   - Average age at entry: {df['age_at_entry_years'].mean():.1f} years")
print(f"   - Most common county: {df['initial_county'].mode()[0]}")

print(f"\n4. DISPARITIES")
highest_longstay_race = race_analysis['Long Stay Rate'].idxmax()
print(f"   - Highest long stay rate: {highest_longstay_race} ({race_analysis.loc[highest_longstay_race, 'Long Stay Rate']:.1f}%)")

print(f"\n5. REMOVAL REASONS")
print(f"   - Most common: {removal_reasons.index[0]} ({removal_reasons.iloc[0]} cases)")

print("\n" + "="*70)

## Next Steps

1. **Multilevel modeling**: Analyze nested structure (children within counties)
2. **Survival analysis**: Model time to exit using Cox proportional hazards
3. **NLP on case notes**: Extract themes and keywords
4. **Power BI dashboard**: Create interactive visualizations
5. **Policy simulations**: Model impact of interventions